In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt

In [19]:
raw = pd.read_csv("./marketing_campaign.csv")

## 결측 제거

In [20]:
drop_na = raw.dropna(axis=0)

In [21]:
## Z_ 변수 제거

drop_col = drop_na.drop(columns = ['Z_CostContact','Z_Revenue'])

## +)

In [22]:
## age 추가

drop_col['age'] = 2021 - drop_col['Year_Birth'] 

In [23]:
## 날짜 format

drop_col['Dt_Customer'] = pd.to_datetime(drop_col['Dt_Customer'],format = '%d-%m-%Y')

## 이상치 제거

In [24]:
def outlier_iqr(data):
    q25, q75 = np.quantile(data, 0.25), np.quantile(data, 0.75)
    iqr = q75 - q25
    
    cut_off = iqr*1.5
    lower, upper = q25 - cut_off, q75 + cut_off
    
    print("IQR은", iqr, "이다.")
    print('lower bound 값은',lower,'이다.')
    print("upper bound 값은",upper,'이다.')
    
    # 이상치 구하기 (1시 분위와 4사 분위에 속해있는 데이터 ?? )
    data1 = data[data > upper]
    data2 = data[data < lower]
    
    # 이상치 총 개수 구하기
    print('총 이상치 개수는',data1.shape[0]+data2.shape[0], '이다\n')
    
    return lower, upper

In [40]:
## income 이상치 제거

lower, upper = outlier_iqr(drop_col['Income'])
dr_income_out = drop_col[drop_col['Income'] < upper]

IQR은 33219.0 이다.
lower bound 값은 -14525.5 이다.
upper bound 값은 118350.5 이다.
총 이상치 개수는 8 이다



In [48]:
## age 100살 이상 지우기

dr_out = dr_income_out[dr_income_out['age'] < 100]

## 추가변수 생성

In [50]:
mk = dr_out.copy()

In [51]:
#제품에 지출된 총액
mk['TotalMntSpent'] = mk['MntWines'] + mk['MntFruits'] + mk['MntMeatProducts'] + mk['MntFishProducts'] + mk['MntSweetProducts'] + mk['MntGoldProds']
#총 구매 건수
mk['TotalNumPurchases'] = mk['NumWebPurchases'] + mk['NumCatalogPurchases'] + mk['NumStorePurchases'] + mk['NumDealsPurchases']
# 승인된 총 캠페인 수
mk['Total_Acc_Cmp'] = mk['AcceptedCmp1'] + mk['AcceptedCmp2'] + mk['AcceptedCmp3'] + mk['AcceptedCmp4'] + mk['AcceptedCmp5'] + mk['Response']
# 결혼상태에 따른 파트너 유무
mk["Partner"]=mk["Marital_Status"].replace({"Married":"Yes", "Together":"Yes", "Absurd":"No", "Widow":"No", "YOLO":"No", "Divorced":"No", "Single":"No","Alone":"No"})
# 가구 내 총 자녀 수
mk["Children"] = mk["Kidhome"] +  mk["Teenhome"]
# 가족수
mk["Family_Size"] = mk["Partner"].replace({"No": 1, "Yes":2})+ mk["Children"]
#부모인지 아닌지
mk["Is_Parent"] = np.where(mk.Children> 0, 1, 0)
# 교육 수준을 세 그룹으로 세분화
mk["Education_Level"]=mk["Education"].replace({"Basic":"Undergraduate","2n Cycle":"Undergraduate", "Graduation":"Graduate", "Master":"Postgraduate", "PhD":"Postgraduate"})
# 가입한 날짜, 월 및 연도에 대한 열 추가
mk['Year_Joined'] = mk['Dt_Customer'].dt.year # 연도 추출
mk['Month_Joined'] = mk['Dt_Customer'].dt.strftime("%B") # strftime: 날짜 형식 변환 %B:locale 월
mk['Day_Joined'] = mk['Dt_Customer'].dt.day_name() # day_name: 요일을 문자열로 추출
mk["Dt_Customer"].dt.year.min() , mk["Dt_Customer"].dt.year.max()
# 연도 생성_고객
mk['Years_Customer'] = (mk["Dt_Customer"].dt.year.max()) - (mk['Dt_Customer'].dt.year)
mk['Days_Customer'] = (mk["Dt_Customer"].max()) - (mk['Dt_Customer'])
# 나이에 따른 그룹 나누기
mk['Age_Group'] = pd.cut(x = mk['age'], bins = [1, 17, 24, 44, 64, 150],
                         labels = ['Under 18','Young adult','Adult','Middel Aged','Senior Citizen'])

## 차원 축소

In [55]:
# 더미변수까지 만들면 ...
dum = pd.get_dummies(mk)

In [62]:
pd.set_option('display.max_columns', None) ## 변수 77개... 너무 많은 느낌! 차원 축소 필요

### 방법1. 유사한 범주들을 합쳐서 범주의 수 줄이기 (--> 가변수 줄이기)

결혼상태에 따른 파트너 유무 "Partner"  
교육 수준을 세 그룹으로 세분화 "Education_Level"  
로 묶고, 원래 있던 범주형 변수는 제거하기

'Age_Group'는 그냥 age로도 될 거 같음

In [69]:
# Marital_Status 제거 -> Partner로 대체

mk["Marital_Status"].value_counts()

In [ ]:
mk = mk.drop(columns=["Marital_Status"])

In [70]:
# Education -> Educational_Level

mk["Education"].value_counts()

Graduation    1113
PhD            476
Master         364
2n Cycle       198
Basic           54
Name: Education, dtype: int64

In [71]:
mk = mk.drop(columns=["Education"])

In [74]:
# Age_Group
mk = mk.drop(columns=["Age_Group"])

In [79]:
# ID, Year_Birth, Dt_customer는 제거해도 될 듯 (실질적 정보 없음, 이미 다른 추가변수에 녹아든 정보, 처리 어려운 데이터형)
mk = mk.drop(columns=["ID","Year_Birth","Dt_Customer"])

### 방법2. PCA로 연속형 변수 축소

In [76]:
from sklearn.decomposition import PCA
from sklearn import preprocessing

In [97]:
# Days_Customer data type : timedelta -> int
mk['Days_Customer'] = mk['Days_Customer'].dt.days

In [105]:
# 연속형 변수
cont_var = mk.drop(columns=['AcceptedCmp1','AcceptedCmp2','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5',"Complain","Response","Partner","Education_Level","Month_Joined",'Is_Parent','Month_Joined','Day_Joined'])

In [107]:
# 데이터 정규화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(cont_var)
scaled_df = pd.DataFrame(scaler.transform(cont_var))
scaled_df.columns = cont_var.columns
scaled_df

,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,age,Days_Customer,TotalMntSpent,TotalNumPurchases,Total_Acc_Cmp,Children,Family_Size,Year_Joined,Years_Customer
0,0.314651,-0.823405,-0.930767,0.310830,0.974566,1.548614,1.748400,2.449154,1.480301,0.849556,0.361479,1.424772,2.628526,-0.562650,0.689203,1.017189,1.527183,1.679323,1.328161,0.614920,-1.266589,-1.759012,-1.498307,1.498307
1,-0.254877,1.038757,0.906602,-0.380600,-0.874776,-0.638664,-0.731678,-0.652345,-0.635399,-0.735767,-0.168834,-1.132957,-0.588043,-1.179732,-0.139645,1.273530,-1.188629,-0.963897,-1.167390,-0.503808,1.403420,0.448513,1.417639,-1.417639
2,0.965354,-0.823405,-0.930767,-0.795458,0.355155,0.568110,-0.175957,1.336263,-0.149031,-0.039771,-0.699147,1.424772,-0.230646,1.288596,-0.554069,0.333612,-0.205999,0.281242,0.802782,-0.503808,-1.266589,-0.655250,-0.040334,0.040334
3,-1.206087,1.038757,-0.930767,-0.795458,-0.874776,-0.563241,-0.667380,-0.506392,-0.586763,-0.755100,-0.168834,-0.767567,-0.945440,-0.562650,0.274779,-1.289883,-1.060245,-0.920675,-0.904700,-0.503808,0.068415,0.448513,1.417639,-1.417639
4,0.322136,1.038757,-0.930767,1.555404,-0.394659,0.417263,-0.217292,0.150396,-0.003121,-0.561768,1.422105,0.328602,0.126750,0.054432,-0.139645,-1.033542,-0.951612,-0.307248,0.540092,-0.503808,0.068415,0.448513,1.417639,-1.417639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,0.463624,-0.823405,0.906602,-0.104028,1.193879,0.417263,0.076644,0.077420,2.209853,3.923536,-0.168834,1.790162,0.126750,-0.562650,-0.139645,0.162718,0.134712,1.220500,0.408747,-0.503808,0.068415,0.448513,-0.040334,0.040334
2201,0.598401,2.900920,0.906602,0.241687,0.295881,-0.663806,-0.621452,-0.688833,-0.659718,-0.697100,2.482731,1.424772,-0.230646,-0.254109,0.689203,1.957107,-1.652786,-0.270675,0.934126,0.614920,2.738424,2.656038,1.417639,-1.417639
2202,0.258780,-0.823405,-0.930767,1.451690,1.783653,0.542969,0.237389,-0.105022,-0.367897,-0.387769,-0.699147,-0.767567,0.126750,2.214218,0.274779,-1.033542,-0.981239,1.054260,0.540092,0.614920,-1.266589,-1.759012,1.417639,-1.417639
2203,0.851004,-0.823405,0.906602,-1.417746,0.361082,0.090428,0.223611,0.770696,0.069834,0.327560,-0.168834,0.693992,0.841543,1.288596,-0.968493,1.102636,-0.976302,0.392623,1.065471,-0.503808,0.068415,0.448513,1.417639,-1.417639


In [108]:
# PCA 적용
pcs = PCA()
pcs.fit(scaled_df.dropna(axis=0))
pcsSummary_df = pd.DataFrame({'Standard deviation': np.sqrt(pcs.explained_variance_),
                           'Proportion of variance': pcs.explained_variance_ratio_,
                           'Cumulative proportion': np.cumsum(pcs.explained_variance_ratio_)})
pcsSummary_df = pcsSummary_df.transpose()
pcsSummary_df.columns = ['PC{}'.format(i) for i in range(1, len(pcsSummary_df.columns) + 1)]
pcsSummary_df.round(4)

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24
Standard deviation,2.9607,1.8402,1.6039,1.0956,1.0916,1.0036,0.9908,0.8520,0.7944,0.7559,0.6853,0.6571,0.6354,0.6159,0.5812,0.5282,0.4804,0.4389,0.3723,0.3377,0.0,0.0,0.0,0.0
Proportion of variance,0.3651,0.1410,0.1071,0.0500,0.0496,0.0419,0.0409,0.0302,0.0263,0.0238,0.0196,0.0180,0.0168,0.0158,0.0141,0.0116,0.0096,0.0080,0.0058,0.0047,0.0,0.0,0.0,0.0
Cumulative proportion,0.3651,0.5061,0.6132,0.6632,0.7129,0.7548,0.7957,0.8259,0.8522,0.8760,0.8956,0.9135,0.9304,0.9462,0.9602,0.9718,0.9815,0.9895,0.9953,1.0000,1.0,1.0,1.0,1.0


In [109]:
# PC12의 Cumulative proportion = 96% (즉, 전체 변동의 96% 설명 가능) --> 15개 주성분으로.. 

In [111]:
# 성분이 각 변수에 가지는 가중치를 확인 가능

pcsComponents_df = pd.DataFrame(pcs.components_.transpose(), columns=pcsSummary_df.columns, 
                                index=scaled_df.columns)
pcsComponents_df.iloc[:,:5]

,PC1,PC2,PC3,PC4,PC5
Income,0.286999,-0.029373,0.175072,-0.004640,-0.020931
Kidhome,-0.232402,0.058705,0.023267,0.429455,0.265690
Teenhome,-0.062577,0.237886,0.437498,-0.120808,-0.231985
Recency,0.003176,0.020177,0.002767,0.184135,-0.331838
MntWines,0.267850,0.096301,0.095431,-0.190176,0.216970
MntFruits,0.228930,-0.047898,-0.030457,0.338240,-0.117130
MntMeatProducts,0.276969,-0.056252,-0.045448,0.139255,0.036066
MntFishProducts,0.238166,-0.049992,-0.043469,0.316433,-0.132842
MntSweetProducts,0.230007,-0.034501,-0.025414,0.317723,-0.113014
MntGoldProds,0.194057,0.084286,0.026500,0.118694,0.011169


In [118]:
# 12개 주성분의 값을 가져가기
principalComponents = pd.DataFrame(data = pcs.transform(scaled_df), columns = ['PC{}'.format(i) for i in range(1, len(pcsSummary_df.columns) + 1)])
principalComponents

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24
0,5.106229,1.814954,-2.087767,0.357088,-0.233016,0.100271,-0.056083,1.475788,2.127977,0.250317,0.157495,-0.234628,1.529596,0.376081,0.494701,-0.805936,-0.095010,-0.492692,0.175895,0.082478,-6.705006e-16,9.432795e-16,-1.036187e-16,8.447483e-16
1,-3.269537,-1.674840,1.928414,-0.275166,-0.540859,-0.914275,0.589725,0.453310,0.495244,0.390935,-0.017047,0.036624,0.009218,0.209619,-0.025028,0.049134,-0.246223,-0.715827,-0.006252,-0.196506,5.530246e-16,4.075102e-16,1.169932e-16,-3.919091e-16
2,2.296211,-0.611505,-0.284261,-0.574988,-0.352523,1.372397,0.792472,-0.652905,0.242720,-0.386420,-1.334301,0.045443,-0.097817,-1.136168,0.534297,0.217444,-0.299940,0.341047,-0.370344,0.135505,-4.707540e-16,-4.155295e-17,3.876085e-16,6.240032e-16
3,-3.051460,-2.245473,-0.048772,0.715441,1.191585,0.731678,0.387707,-0.460012,-0.052812,0.145805,-0.202392,0.049295,0.013373,-0.102254,-0.183425,-0.117209,0.080725,0.368507,0.244010,-0.236572,1.205281e-15,4.075102e-16,1.169932e-16,-7.180371e-16
4,-0.503339,-1.244840,1.290404,1.947645,0.758766,1.090418,-1.725363,-0.586047,0.777375,0.336829,0.192796,-0.150378,-0.237258,-0.128852,0.581954,-0.023276,-0.404128,0.273782,-0.528664,-0.370944,2.581216e-16,2.964879e-16,1.169932e-16,-2.045589e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,2.619992,1.053365,1.106019,0.663821,-0.335917,0.925875,0.349950,2.874708,-1.577796,-0.972386,-0.627647,1.249408,0.197620,1.609935,-1.136259,0.311379,-0.772661,0.452904,0.393073,-0.065102,4.836197e-17,-9.706410e-17,-5.838366e-16,7.312308e-16
2201,-2.307446,1.207827,5.286067,0.830156,1.884090,-0.833970,-0.667419,0.316069,2.054078,0.602212,-1.023566,0.497635,-0.585118,-0.043632,0.613625,0.180565,-0.435555,0.042865,-0.315146,0.287061,2.997550e-16,6.295548e-16,2.280155e-16,-6.625259e-16
2202,2.169336,-2.498320,-0.135676,-0.646006,0.948447,0.773094,-2.222500,-1.258011,-0.286042,-0.176778,0.014525,-1.291774,-0.753324,-0.973197,-1.688567,-0.453148,0.238000,-0.511548,0.213807,-0.174736,-1.018635e-15,7.444332e-17,2.280155e-16,-4.266036e-16
2203,1.833444,-1.191651,2.866393,-0.588794,-0.306730,0.609806,1.417514,-0.293447,-0.144980,0.334536,-0.395669,0.191080,0.282854,-0.392848,0.046301,-0.436908,0.335709,0.150592,0.055157,-0.292254,-5.780151e-16,4.075102e-16,2.280155e-16,6.628028e-16


In [122]:
Take_Component = principalComponents.iloc[:,0:15]

In [124]:
# 원 데이터프레임의 범주형 데이터들과 합치기
cate_var = mk[['AcceptedCmp1','AcceptedCmp2','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5',"Complain","Response","Partner","Education_Level","Month_Joined",'Is_Parent','Month_Joined','Day_Joined']]
final_df = pd.concat([cate_var, Take_Component], axis=1)
final_df

,AcceptedCmp1,AcceptedCmp2,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,Complain,Response,Partner,Education_Level,Month_Joined,Is_Parent,Month_Joined,Day_Joined,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15
0,0,0,0,0,0,0,1,No,Graduate,September,0,September,Tuesday,5.106229,1.814954,-2.087767,0.357088,-0.233016,0.100271,-0.056083,1.475788,2.127977,0.250317,0.157495,-0.234628,1.529596,0.376081,0.494701
1,0,0,0,0,0,0,0,No,Graduate,March,1,March,Saturday,-3.269537,-1.674840,1.928414,-0.275166,-0.540859,-0.914275,0.589725,0.453310,0.495244,0.390935,-0.017047,0.036624,0.009218,0.209619,-0.025028
2,0,0,0,0,0,0,0,Yes,Graduate,August,0,August,Wednesday,2.296211,-0.611505,-0.284261,-0.574988,-0.352523,1.372397,0.792472,-0.652905,0.242720,-0.386420,-1.334301,0.045443,-0.097817,-1.136168,0.534297
3,0,0,0,0,0,0,0,Yes,Graduate,February,1,February,Monday,-3.051460,-2.245473,-0.048772,0.715441,1.191585,0.731678,0.387707,-0.460012,-0.052812,0.145805,-0.202392,0.049295,0.013373,-0.102254,-0.183425
4,0,0,0,0,0,0,0,Yes,Postgraduate,January,1,January,Sunday,-0.503339,-1.244840,1.290404,1.947645,0.758766,1.090418,-1.725363,-0.586047,0.777375,0.336829,0.192796,-0.150378,-0.237258,-0.128852,0.581954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,0,0,0,0,0,0,0,Yes,Graduate,June,1,June,Thursday,2.619992,1.053365,1.106019,0.663821,-0.335917,0.925875,0.349950,2.874708,-1.577796,-0.972386,-0.627647,1.249408,0.197620,1.609935,-1.136259
2201,1,0,0,0,0,0,0,Yes,Postgraduate,June,1,June,Tuesday,-2.307446,1.207827,5.286067,0.830156,1.884090,-0.833970,-0.667419,0.316069,2.054078,0.602212,-1.023566,0.497635,-0.585118,-0.043632,0.613625
2202,0,0,0,1,0,0,0,No,Graduate,January,0,January,Saturday,2.169336,-2.498320,-0.135676,-0.646006,0.948447,0.773094,-2.222500,-1.258011,-0.286042,-0.176778,0.014525,-1.291774,-0.753324,-0.973197,-1.688567
2203,0,0,0,0,0,0,0,Yes,Postgraduate,January,1,January,Friday,1.833444,-1.191651,2.866393,-0.588794,-0.306730,0.609806,1.417514,-0.293447,-0.144980,0.334536,-0.395669,0.191080,0.282854,-0.392848,0.046301


28개의 변수로 차원축소 성공

In [125]:
# 더미변수까지 하면? - 59개.. 나쁘지 않은듯 ?
dum = pd.get_dummies(final_df)
dum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2205 entries, 0 to 2204
Data columns (total 59 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   AcceptedCmp1                   2205 non-null   int64  
 1   AcceptedCmp2                   2205 non-null   int64  
 2   AcceptedCmp3                   2205 non-null   int64  
 3   AcceptedCmp4                   2205 non-null   int64  
 4   AcceptedCmp5                   2205 non-null   int64  
 5   Complain                       2205 non-null   int64  
 6   Response                       2205 non-null   int64  
 7   Is_Parent                      2205 non-null   int64  
 8   PC1                            2205 non-null   float64
 9   PC2                            2205 non-null   float64
 10  PC3                            2205 non-null   float64
 11  PC4                            2205 non-null   float64
 12  PC5                            2205 non-null   f

++)   
나중에 pca과정 시각화 할 때  
https://plotly.com/python/pca-visualization/  
여기 참고해서 만들면 깐지 날 듯

### 방법3. 회귀분석으로 범주형 변수 축소
좀 더 공부 필요 ~! 그리고 굳이 해야하나 ? 59개 많은건가 ??